In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

!pip install -q kaggle
!mkdir -p ~/.kaggle
with open("kaggle.json", 'w+') as f:
        f.write('{"username":"amulchoudhary","key":"46d30673f152a671757ee4f2acc2c158"}')
!cp kaggle.json ~/.kaggle/

from google.colab import drive
drive.mount('/content/drive')

In [4]:
!kaggle datasets download -d drscarlat/dzd-data
!unzip dzd-data.zip

dzd-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dzd-data.zip
  inflating: dataset.npy             


In [0]:
DataRaw = np.load('./dataset.npy', allow_pickle=True)[()]
data_x, data_y = DataRaw['genes'], DataRaw['resistant']

In [0]:
max_words = 64
maxlen = 54

Codons = list(data_x)

for n in range(len(Codons)):
    Codons[n] = list([Codons[n][i:i+3] for i in range(0, len(Codons[n]), 3)])

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=True)
tokenizer.fit_on_texts(Codons)

sequences = tokenizer.texts_to_sequences(Codons)
word_index = tokenizer.word_index
Xpad = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post', value=0)

data_x = Xpad

In [0]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.1)

In [91]:
train_y.shape

(90000,)

# MODEL

In [93]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Embedding, LSTM
from keras import regularizers, layers, preprocessing

model = Sequential()
model.add(Embedding(64, 1, input_length=maxlen))
model.add(layers.Conv1D(128, 27, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Dropout(0.5))
model.add(layers.Conv1D(128, 9, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Bidirectional(layers.GRU(32, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Using TensorFlow backend.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 54, 1)             64        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 28, 128)           3584      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 9, 128)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 128)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 128)            147584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 128)            0         
________________________

In [94]:
model.fit(train_x, train_y, epochs = 10, validation_split=0.2)


Train on 72000 samples, validate on 18000 samples
Epoch 1/10




72000/72000 [==============================] - 59s 818us/step - loss: 0.0865 - acc: 0.9685 - val_loss: 0.0443 - val_acc: 0.9877
Epoch 2/10
72000/72000 [==============================] - 56s 781us/step - loss: 0.0431 - acc: 0.9882 - val_loss: 0.0425 - val_acc: 0.9878
Epoch 3/10
72000/72000 [==============================] - 57s 786us/step - loss: 0.0399 - acc: 0.9884 - val_loss: 0.0400 - val_acc: 0.9879
Epoch 4/10
72000/72000 [==============================] - 56s 780us/step - loss: 0.0393 - acc: 0.9884 - val_loss: 0.0393 - val_acc: 0.9879
Epoch 5/10
72000/72000 [==============================] - 56s 776us/step - loss: 0.0382 - acc: 0.9885 - val_loss: 0.0388 - val_acc: 0.9879
Epoch 6/10
72000/72000 [==============================] - 56s 782us/step - loss: 0.0385 - acc: 0.9882 - val_loss: 0.0387 - val_acc: 0.9879
Epoch 7/10
72000/72000 [==============================] - 56s 777us/step - loss: 0.0380 - acc: 0.9883 - val_los

In [95]:
model.evaluate(test_x, test_y)

10000/10000 [==============================] - 1s 85us/step


[0.03600967006910359, 0.9888]